In [1]:
%matplotlib inline

In [2]:
from geo_tool import Point_Cloud
import numpy as np
import matplotlib.pylab as plt
import tensorflow as tf
from tf_lab.external.structural_pc_losses import losses

In [3]:
nn_distance, approx_match, match_cost = losses()

In [4]:
# all_pcs = '/orions4-zfs/projects/optas/DATA/OUT/iclr/synthetic_samples/voxel_based/gmm/ae_chair_sn_vox_data_32_voxels_64_bneck/samples_with_lcc.npz'
# all_pcs = np.load(all_pcs)
# all_pcs = all_pcs[all_pcs.keys()[0]]

In [4]:
n_pc_1 = 100000
n_pc_2 = 100000
pc_1 = tf.Variable(np.random.randn(50, n_pc_1, 3).astype('float32'))
pc_2 = tf.Variable(np.random.randn(50, n_pc_2, 3).astype('float32'))

In [5]:
match = approx_match(pc_1, pc_2)
emd_loss = match_cost(pc_1, pc_2, match)

In [6]:
cost_p1_p2, _, cost_p2_p1, _ = nn_distance(pc_1, pc_2)
cd_loss = tf.reduce_mean(cost_p1_p2) + tf.reduce_mean(cost_p2_p1)

In [7]:
sess = tf.InteractiveSession()
init = tf.global_variables_initializer()
sess.run(init)

In [8]:
a = sess.run(cd_loss)

In [9]:
# Let's make a big AE.

0.009552474

In [11]:
from tf_lab.point_clouds.autoencoder import Configuration as Conf
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.ae_templates import mlp_architecture_ala_iclr_18, default_train_params

In [12]:
import sys
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt
from functools import partial

from general_tools.notebook.tf import reset_tf_graph
from general_tools.in_out import create_dir
from general_tools.in_out.basics import create_dir, delete_files_in_directory, files_in_subdirs

from geo_tool import Point_Cloud

from tf_lab.in_out.basics import Data_Splitter
from tf_lab.point_clouds.ae_templates import mlp_architecture_ala_iclr_18, default_train_params

from tf_lab.point_clouds.autoencoder import Configuration as Conf
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder

from tf_lab.point_clouds.in_out import load_point_clouds_from_filenames, PointCloudDataSet
from tf_lab.data_sets.shape_net import pc_loader as snc_loader
from tf_lab.data_sets.shape_net import snc_category_to_synth_id

from tf_lab.iclr.helper import load_multiple_version_of_pcs, find_best_validation_epoch_from_train_stats

In [20]:
# class_name = 'chair'
# syn_id = snc_category_to_synth_id()[class_name]
# in_data = load_multiple_version_of_pcs('uniform_one' ,syn_id, n_classes=1, n_pc_points=n_pc_points)

In [49]:
n_pc_points = 100000
batch_size = 50


train_params = default_train_params()
bneck = 128
experiment_id = 'test'
train_dir = 'skata'
create_dir(train_dir)
reset_tf_graph()  

encoder, decoder, enc_args, dec_args = mlp_architecture_ala_iclr_18(n_pc_points, bneck)
loss = 'chamfer'
conf = Conf(n_input = [n_pc_points, 3],
            loss = loss,
                training_epochs = 500,
                batch_size = 1,
                denoising = False,
                learning_rate = train_params['learning_rate'],
                train_dir = train_dir,
                loss_display_step = 1,
                saver_step = None,
                z_rotate = False,
                encoder = encoder,
                decoder = decoder,
                encoder_args = enc_args,
                decoder_args = dec_args
               )
print conf
conf.experiment_name = 'experiment_' + str(experiment_id)
#     conf.held_out_step = 5
#     conf.save(osp.join(train_dir, 'configuration'))
ae = PointNetAutoEncoder(conf.experiment_name, conf)

                    batch_size: 1
                 consistent_io: None
                         debug: False
                       decoder: decoder_with_fc_only
                  decoder_args: {'b_norm_finish': False, 'verbose': True, 'b_norm': False, 'layer_sizes': [256, 256, 300000]}
                       encoder: encoder_with_convs_and_symmetry_new
                  encoder_args: {'filter_sizes': [1], 'n_filters': [64, 128, 128, 256, 128], 'verbose': True, 'b_norm': True, 'strides': [1]}
                 gauss_augment: None
                  is_denoising: False
               latent_vs_recon: 1.0
                 learning_rate: 0.0005
                          loss: chamfer
             loss_display_step: 1
                       n_input: [100000, 3]
                      n_output: [100000, 3]
                           n_z: None
             saver_max_to_keep: None
                    saver_step: None
                     train_dir: skata
               training_epochs: 500
     

In [50]:
train_data = np.random.randn(4*batch_size, n_pc_points, 3).astype('float32')
train_data = PointCloudDataSet(train_data)

In [48]:
ae._single_epoch_train(train_data, conf)

(0.030748368785716594, 15.788946866989136)